In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

In [2]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"

In [3]:
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [4]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [6]:
y = data['quality']

In [8]:
X = data.drop(['quality'], axis = 1)

In [10]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.3, random_state = 17)

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

# Linear regression

In [14]:
linreg = LinearRegression().fit(X_train_scaled, y_train)

**Question 1: What are mean squared errors of model predictions on train and holdout sets?**

In [19]:
y_pred_train = linreg.predict(X_train_scaled)

In [15]:
y_pred = linreg.predict(X_holdout_scaled)

In [54]:
print('Mean squared error (train):%.3f' %mean_squared_error(y_train,y_pred_train))
print('Mean squared error (test):%.3f' %mean_squared_error(y_holdout,y_pred))

Mean squared error (train):0.558
Mean squared error (test):0.584


**Question 2: Which feature this linear regression model treats as the most influential on wine quality?**

In [37]:
linreg_coef = pd.DataFrame({'coef': linreg.coef_, 'influence' : np.abs(linreg.coef_)},
                           index = X.columns)
linreg_coef.sort_values(by = 'influence', ascending=False)

,coef,influence
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
fixed acidity,0.097822,0.097822
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127


# Lasso regression

**Train a LASSO model with alpha = 0.01 (weak regularization) and scaled data. Again, set random_state=17**

In [39]:
lasso1 = Lasso(alpha = 0.01).fit(X_train_scaled, y_train)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [40]:
lasso1_coef = pd.DataFrame({'coef': lasso1.coef_, 'informative' : np.abs(lasso1.coef_)},
                           index = X.columns)
lasso1_coef.sort_values(by = 'informative', ascending=True)

,coef,informative
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000
total sulfur dioxide,-0.000000,0.000000
chlorides,-0.002747,0.002747
sulphates,0.029722,0.029722
free sulfur dioxide,0.043088,0.043088
pH,0.067277,0.067277
volatile acidity,-0.188479,0.188479
density,-0.235492,0.235492
residual sugar,0.256363,0.256363


**Train LassoCV with random_state=17 to choose the best value of alpha in 5-fold cross-validation**

In [41]:
alphas = np.logspace(-6, 2, 200)

In [61]:
lasso_cv = LassoCV(cv=5, alphas = alphas, random_state=17).fit(X_train_scaled, y_train)

In [62]:
lasso_cv.alpha_

0.0002833096101839324

**Question 3: Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [63]:
lasso_cv_coef = pd.DataFrame({'coef': lasso_cv.coef_, 'informative' : np.abs(lasso_cv.coef_)},
                           index = X.columns)
lasso_cv_coef.sort_values(by = 'informative', ascending=True)

,coef,informative
citric acid,-0.000000,0.000000
chlorides,0.006933,0.006933
total sulfur dioxide,0.012969,0.012969
free sulfur dioxide,0.042698,0.042698
sulphates,0.060939,0.060939
fixed acidity,0.093295,0.093295
alcohol,0.137115,0.137115
pH,0.146549,0.146549
volatile acidity,-0.192049,0.192049
residual sugar,0.526883,0.526883


**Question 4: What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [53]:
print('Mean squared error (train):%.3f' %mean_squared_error(y_train,lasso_cv.predict(X_train_scaled)))
print('Mean squared error (test):%.3f' %mean_squared_error(y_holdout,lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train):0.558
Mean squared error (test):0.583


# Random Forest

In [60]:
forest = RandomForestRegressor(random_state=17).fit(X_train_scaled, y_train)

In [68]:
print('Mean squared error (train): %.3f' %mean_squared_error(y_train,forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" %(np.abs(cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error'))).mean())
print('Mean squared error (test): %.3f' %mean_squared_error(y_holdout,forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


**Tune the max_features and max_depth hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [70]:
forest_params = {'max_depth': list(range(10, 25)),
                'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest, param_grid = forest_params, n_jobs=-1)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(estimator=RandomForestRegressor(random_state=17), n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11]})

In [71]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6}, 0.4939384585901731)

**Question 6: What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring=’neg_mean_squared_error’ and other arguments left with default values) and on holdout set?**

In [72]:
tuned_forest = locally_best_forest.best_estimator_

In [73]:
print('Mean squared error (train): %.3f' %mean_squared_error(y_train,tuned_forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" %(np.abs(cross_val_score(tuned_forest, X_train_scaled, y_train, scoring='neg_mean_squared_error'))).mean())
print('Mean squared error (test): %.3f' %mean_squared_error(y_holdout,tuned_forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.054
Mean squared error (cv): 0.398
Mean squared error (test): 0.366


**Question 7: What is the most important feature, according to the Random Forest model?**

In [75]:
rf_importance = pd.DataFrame({'importance': tuned_forest.feature_importances_},
                           index = X.columns)
rf_importance.sort_values(by = 'importance', ascending=False)

,importance
alcohol,0.206056
volatile acidity,0.117578
free sulfur dioxide,0.111556
density,0.088549
pH,0.073659
total sulfur dioxide,0.073640
chlorides,0.073366
residual sugar,0.072072
citric acid,0.062601
fixed acidity,0.061813
